In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
import os
print("Current working directory:")
print(os.getcwd())

Current working directory:
c:\Users\saifu\OneDrive - American International University-Bangladesh\Desktop\10th Semester\CVPR\Projects\CVPR\Final\Assignment_1


In [3]:
import os

print("Does 'Models' folder exist?", os.path.isdir("Models"))

if os.path.isdir("Models"):
    print("Files inside Models:")
    print(os.listdir("Models"))


Does 'Models' folder exist? True
Files inside Models:
['mnist_digit_model.keras']


In [4]:
import os

for root, dirs, files in os.walk(os.getcwd()):
    for f in files:
        if "mnist" in f.lower():
            print(os.path.join(root, f))

c:\Users\saifu\OneDrive - American International University-Bangladesh\Desktop\10th Semester\CVPR\Projects\CVPR\Final\Assignment_1\Models\mnist_digit_model.keras


In [6]:
MODEL_PATH = "Models\mnist_digit_model.keras"
THRESHOLD = 0.70

model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded:", MODEL_PATH)

Model loaded: Models\mnist_digit_model.keras


In [8]:
def preprocess_roi(roi_bgr):
    roi_gray = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2GRAY)
    roi_gray = cv2.GaussianBlur(roi_gray, (5, 5), 0)

    # Try INV MNIST-like 
    _, th = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find digit contour
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        x = np.zeros((1, 784), dtype=np.float32)
        return x, np.zeros((28, 28), dtype=np.uint8), False

    c = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(c)

    # Ignore tiny noise
    if area < 200:
        x = np.zeros((1, 784), dtype=np.float32)
        return x, np.zeros((28, 28), dtype=np.uint8), False

    x, y, w, h = cv2.boundingRect(c)

    # Add padding 
    pad = 20
    x0 = max(x - pad, 0)
    y0 = max(y - pad, 0)
    x1 = min(x + w + pad, th.shape[1])
    y1 = min(y + h + pad, th.shape[0])

  
    digit = th[y0:y1, x0:x1]

    # Keep aspect ratio like MNIST style
    H, W = digit.shape
    if H > W:
        new_h = 20
        new_w = max(1, int(W * (20 / H)))
    else:
        new_w = 20
        new_h = max(1, int(H * (20 / W)))

    digit_resized = cv2.resize(digit, (new_w, new_h), interpolation=cv2.INTER_AREA)

    canvas = np.zeros((28, 28), dtype=np.uint8)
    y_off = (28 - new_h) // 2
    x_off = (28 - new_w) // 2
    canvas[y_off:y_off + new_h, x_off:x_off + new_w] = digit_resized

    x_inp = (canvas.astype("float32") / 255.0).reshape(1, 784)
    return x_inp, canvas, True



In [9]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Cannot open webcam.")

print("Press 'q' to quit.")

# A box for the digit
box_size = 150  

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w = frame.shape[:2]
    x1 = w // 2 - box_size // 2
    y1 = h // 2 - box_size // 2
    x2 = x1 + box_size
    y2 = y1 + box_size

    roi = frame[y1:y2, x1:x2]

    x_inp, roi_28, ok = preprocess_roi(roi)

    if ok:
        probs = model.predict(x_inp, verbose=0)[0]
        pred = int(np.argmax(probs))
        conf = float(np.max(probs))
    else:
        pred, conf = None, 0.0


    # Threshold logic
    if conf >= THRESHOLD:
        text = f"Digit: {pred}  Conf: {conf:.2f}"
        color = (0, 255, 0)
    else:
        text = f"Uncertain (max={conf:.2f} < {THRESHOLD})"
        color = (0, 0, 255)

    # ROI box + prediction
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    cv2.putText(frame, text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    # What model sees (28x28)
    roi_preview = cv2.resize(roi_28, (140, 140), interpolation=cv2.INTER_NEAREST)
    roi_preview = cv2.cvtColor(roi_preview, cv2.COLOR_GRAY2BGR)
    frame[10:150, w-150:w-10] = roi_preview
    cv2.putText(frame, "Model Input", (w-150, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1)

    cv2.imshow("MNIST Webcam Digit Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


RuntimeError: Cannot open webcam.